In [ ]:
# 텐서플로 2 버전 선택
try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split #데이터셋을 나누는 함수
from keras.models import load_model

In [ ]:
#데이터 불러오기
x = np.load("/content/drive/My Drive/numpy_data/savedfp1_x_z.npy")  #saved2_x.npy, saved3_x.npy
y = np.load("/content/drive/My Drive/numpy_data/saved_y.npy")  #saved2_y.npy, saved3_y.npy
x_list = np.array(x)
y_list = np.array(y)

In [ ]:
# 데이터 분리하기
X_train, X_test, Y_train, Y_test = train_test_split(x_list, y_list, test_size=0.2, stratify=y_list) #0.2
X_test, X_val, Y_test, Y_val = train_test_split(X_test, Y_test, test_size=0.5, stratify=Y_test)

In [ ]:
print(np.shape(X_train))
print(np.shape(X_test))
print(np.shape(X_val))

(87, 210000)
(11, 210000)
(11, 210000)


In [ ]:
X_train = np.reshape(X_train, (87, 500, 420)) #(87, 500, 420)
X_test = np.reshape(X_test, (11, 500, 420)) #(11, 500, 420)
X_val = np.reshape(X_val, (11, 500, 420)) #(11, 500, 420)
Y_train = np.reshape(Y_train, (87, 1))
Y_test = np.reshape(Y_test, (11, 1))
Y_val = np.reshape(Y_val, (11, 1))

In [ ]:
model1 = tf.keras.Sequential([
    tf.keras.layers.Conv1D(128, 5, strides=1, input_shape=(500, 420), activation='relu'), #128, 7, 1, (64, 210000)
    tf.keras.layers.MaxPooling1D(2, strides=2), #tf.keras.layers.MaxPooling1D(2, strides=2)
    tf.keras.layers.Conv1D(64, 5, strides=2, activation='relu'), #64, 5, 2
    tf.keras.layers.MaxPooling1D(2, strides=2), #tf.keras.layers.MaxPooling1D(2, strides=2)
    tf.keras.layers.Conv1D(32, 3, strides=1, activation='relu'), #32, 3, 1
    tf.keras.layers.MaxPooling1D(2, strides=2), #2, 2
    tf.keras.layers.LSTM(units=32, return_sequences=True), #32
    tf.keras.layers.LSTM(units=32, return_sequences=True), #32
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=10, activation='relu'), #10
    tf.keras.layers.Dropout(rate=0.2), #0.2
    tf.keras.layers.Dense(units=2, activation='softmax') #2
])

model1.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])
#model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.Accuracy()]) #binary_crossentropy
model1.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_28 (Conv1D)           (None, 496, 128)          268928    
_________________________________________________________________
max_pooling1d_27 (MaxPooling (None, 248, 128)          0         
_________________________________________________________________
conv1d_29 (Conv1D)           (None, 122, 64)           41024     
_________________________________________________________________
max_pooling1d_28 (MaxPooling (None, 61, 64)            0         
_________________________________________________________________
conv1d_30 (Conv1D)           (None, 59, 32)            6176      
_________________________________________________________________
max_pooling1d_29 (MaxPooling (None, 29, 32)            0         
_________________________________________________________________
lstm_18 (LSTM)               (None, 29, 32)           

In [ ]:
model2 = tf.keras.Sequential([
    tf.keras.layers.LSTM(units=64, input_shape=(500, 420), return_sequences=True), #32
    tf.keras.layers.LSTM(units=32, return_sequences=True), #32
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=10, activation='LeakyReLU'), #10
    tf.keras.layers.Dropout(rate=0.2), #0.2
    tf.keras.layers.Dense(units=2, activation='softmax') #2
])

model2.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])
#model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.Accuracy()]) #binary_crossentropy
model2.summary()

Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_51 (LSTM)               (None, 500, 64)           124160    
_________________________________________________________________
lstm_52 (LSTM)               (None, 500, 32)           12416     
_________________________________________________________________
flatten_21 (Flatten)         (None, 16000)             0         
_________________________________________________________________
dense_45 (Dense)             (None, 10)                160010    
_________________________________________________________________
dropout_24 (Dropout)         (None, 10)                0         
_________________________________________________________________
dense_46 (Dense)             (None, 2)                 22        
Total params: 296,608
Trainable params: 296,608
Non-trainable params: 0
_______________________________________________

In [ ]:
history = model.fit(X_train, Y_train, epochs=17, batch_size=64, validation_data=(X_val, Y_val)) #epoch=17 batch_size=64

plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], 'b-', label='loss')
plt.plot(history.history['val_loss'], 'r--', label='val_loss')
plt.xlabel('Epoch')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['sparse_categorical_accuracy'], 'g-', label='accuracy')
plt.plot(history.history['val_sparse_categorical_accuracy'], 'k--', label='val_accuracy')
plt.xlabel('Epoch')
plt.legend()

plt.show()

In [ ]:
# 테스트 정확도 출력
#print("\n Test Accuracy: %.4f" % (model.evaluate(X_test, Y_test)[1]))
score = model.evaluate(X_test, Y_test, verbose=0)
print("정답률: ", score[1], "loss: ", score[0])

In [ ]:
test_p = model.predict(X_train)
print(Y_train.flatten())
print(test_p)

In [ ]:
from sklearn.datasets import make_regression
xp, a = make_regression(n_samples=11, n_features=2, noise=0.1, random_state=1)